In [1]:
import numpy as np

In [2]:
#read the pdb file
#save atom name and coordinates
#generate atom type
#save the list of H on two face seperately and Si that bonded to H
name = ['',]
atp = ['',]
dummystr = ['',]
x = [0,]
y = [0,]
z = [0,]
charge=[0,]
mass=[0,]
N = 3200
b = [[] for i in range(N+1)]
inf = open('Hsilica.pdb','r')
htop = []
hbot = []

for line in inf:
    if line.split()[0] == 'HETATM':
        n = int(line.split()[1])
        name.append(line.split()[2])
        x.append(float(line.split()[5]))
        y.append(float(line.split()[6]))
        z.append(float(line.split()[7]))
        if name[n] == 'H':
            atp.append('HOY')
            charge.append(0.4)
            mass.append(1.008)
            dummystr.append('H+')
            if z[n] > 7 :
                htop.append(n)
            else:
                hbot.append(n)
        
        if name[n] == 'SI':
            atp.append('SC4')
            charge.append(1.1)
            mass.append(28.086)
            dummystr.append('Si4+')
        if name[n] =='O':
            mass.append(15.9994)
            if (z[n]==15.610)or(z[n]==15.012)or(z[n]==0.602)or(z[n]==1.200):
                atp.append('OC24')
                charge.append(-0.675)
            else:
                atp.append('OC23')
                charge.append(-0.55)
            dummystr.append('O2-')
    if line.split()[0] == 'CONECT':
        n = int(line.split()[1])
        for i in range(2,len(line.split())):
            b[n].append(int(line.split()[i]))
        #print(n,b[n])
               

In [3]:
#Randomly select 18 H from each face
#substitude them with Na+
#delete the bond beteen H and Si
D = 21 # number os H deleted for pH=7
deltop = np.random.choice(htop,D)
delbot = np.random.choice(hbot,D)
for i in range(D):
    d1 = deltop[i]
    d2 = delbot[i]
    
    name[d1] = 'Na'
    atp[d1] = 'NA+'
    dummystr[d1] = 'Na+'
    mass[d1] = 22.9898
    charge[d1] = 1
    z[d1] += 1
    
    name[d2] = 'Na'
    atp[d2] = 'NA+'
    dummystr[d2] = 'Na+'
    mass[d2]=22.9898
    charge[d2]=1
    z[d2] -= 1
    
    for j in b[d1]:
        if (name[j]=='O'):
            b[d1].remove(j)
            b[j].remove(d1)
            charge[j] = -0.9
            for k in b[j]:
                if (name[k]=='SI'):
                    charge[k] = 0.725
                break
            break
            
    for j in b[d2]:     
        if (name[j]=='O'):
            b[d2].remove(j)
            b[j].remove(d2)
            charge[j] = -0.9
            for k in b[j]:
                if (name[k]=='SI'):
                    charge[k] = 0.725
                break
            break

In [4]:
#add trans priodic boundary bonds
"""silica = ('SI','O')
for i in range(1,N):
    if name[i] in silica
    for j in range(i,N+1):"""

"silica = ('SI','O')\nfor i in range(1,N):\n    if name[i] in silica\n    for j in range(i,N+1):"

In [5]:
#print new pdb file
#open writing file

pdb = open('silica.pdb','w')
pdb.write('CRYST1   49.130   49.130   30.212  90.00  90.00 120.00 P1          1\n')
for i in range(1,N+1):
    pdb.write('HETATM %4d %2s   QUA     1     %6.3f  %6.3f  %6.3f  1.00  0.00     %2s\n'%(i,name[i],x[i],y[i],z[i],dummystr[i]))
pdb.write('END')
pdb.close()


In [6]:
#print itp file
angle_type = ['OC23SC4OC23','OC24SC4OC24','OC23SC4OC24','SC4OC23SC4','SC4OC24SC4','SC4OC23HOY','SC4OC24HOY']
#open writing file
itp = open('silica.itp','w')
#print moleculetype
itp.write('[ moleculetype ]\n;name            nrexcl\n QUA             1\n\n')
#print atom type
itp.write('[ atoms ]\n;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type\n')
for i in range(1,N+1):
     itp.write("%4d     %4s  1  QUA    %2s  %4d      %f  %f\n"%(i,atp[i],name[i],i,charge[i],mass[i]))
itp.write('\n')
#print bonds
itp.write('[ bonds ]\n;   ai     aj funct   r             k[ bonds ]\n')
for i in range(1,N+1):
    for j in b[i]:
        if j>i :
            if (atp[i]=='SC4')or(atp[j]=='SC4'):
                b0 = 0.168
                kb = 238488
            else:
                b0 = 0.0945
                kb = 414216
            itp.write(' %4d %4d      1       %f   %f\n'%(i,j,b0,kb))
itp.write('\n')
#print angles
itp.write('[ pairs ]\n\n[ angles ]\n;  i    j    k  func       th0       cth\n')
for i in range(1,N+1):
    for j in b[i]:
        for k in b[j]:
            if (i<k):
                if (atp[j]=='SC4'):
                    th0 = 109.5
                    cth = 836.8
                elif (atp[i]=='SC4')and(atp[k]=='SC4'):
                    th0 = 149.0
                    cth = 836.8
                else:
                    th0 = 115
                    cth = 418.4
                itp.write(' %4d %4d %4d      5      %4f    %1f  0.0  0.0\n'%(i,j,k,th0,cth))
                if not((atp[i]+atp[j]+atp[k])in angle_type or (atp[k]+atp[j]+atp[i])in angle_type):
                    print('error',i,j,k,atp[i],atp[j],atp[k])
itp.write('[ dihedrals ]\n')

itp.close()